In [21]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime


def FetchData(date="2025-03-21"):
    DIR = "HEADERS.json"
    URL = "https://dps.psx.com.pk/historical"

    with open(DIR, "r") as file: headers = json.load(file)
    payload = {"date": date}
    response = requests.post(URL, headers=headers, data=payload)

    return response

In [22]:
def preprocessing(html_response):
    if html_response.status_code == 200:
        soup = BeautifulSoup(html_response.text, "html.parser")
    else:
        soup = None
    table = soup.find("table")
    headers = [header.text.strip() for header in table.find_all("th")]

    rows = []
    for row in table.find_all("tr")[1:]:  # Skip header row
        cells = [cell.text.strip() for cell in row.find_all("td")]
        rows.append(cells)
    df = pd.DataFrame(rows, columns=headers)
    return df

In [43]:
date = "2025-03-27"
response = FetchData(date)
df = preprocessing(response)
current_time = datetime.now().time()
df.to_csv(date+".csv")
print('Data Fetched Successfully')

Data Fetched Successfully


**Additional Metrics**

In [46]:
import requests

url = "https://dps.psx.com.pk/screener"
DIR = "HEADERS.json"
with open(DIR, "r") as file: headers = json.load(file)
response = requests.get(url, headers=headers)
df_metric_new = preprocessing(response)

In [52]:
merged_df = pd.merge(df, df_metric_new, on="SYMBOL", how="left")
final_df = merged_df[['SYMBOL', 'LDCP', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'CHANGE',
       'CHANGE (%)_x', 'VOLUME', 'MARKET CAP.', 'PRICE',
       'CHANGE (%)_y', '1-YEAR CH. (%) *', 'PE RATIO (TTM)',
       'DIVIDEND YIELD (%)', 'FREE FLOAT', '30D VOLUME AVG.']]

In [53]:
final_df

,SYMBOL,LDCP,OPEN,HIGH,LOW,CLOSE,CHANGE,CHANGE (%)_x,VOLUME,MARKET CAP.,PRICE,CHANGE (%)_y,1-YEAR CH. (%) *,PE RATIO (TTM),DIVIDEND YIELD (%),FREE FLOAT,30D VOLUME AVG.
0,AABS,708.23,719.97,748.00,715.00,715.00,6.77,0.96%,"1,245",12.4B,715.00,0.96%,33.02%,20.88,5.41,1.7M,278
1,AATM,43.98,47.00,47.00,45.00,46.51,2.53,5.75%,501,2.1B,46.51,5.75%,2146.86%,25.42,0.00,926.2K,"2,600"
2,ABL,133.50,134.84,134.84,133.01,134.10,0.60,0.45%,"2,554",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABOT,"1,102.50","1,117.98","1,118.91","1,087.00","1,100.35",-2.15,-0.20%,"6,335",107.7B,"1,100.35",-0.19%,143.34%,20.58,0.81,21.1M,"43,127"
4,ACIETF,15.25,15.33,15.36,15.32,15.34,0.09,0.59%,"17,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,YOUW-MAR,3.81,3.78,3.78,3.71,3.73,-0.08,-2.10%,"3,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
650,ZAHID,26.10,27.98,27.98,27.98,26.10,0.00,0.00%,53,5.0B,26.10,0.00%,-3.97%,5.63,0.00,5.9M,663
651,ZAL,15.42,15.90,15.90,15.21,15.26,-0.16,-1.04%,"406,918",4.0B,15.26,-1.04%,-1.04%,0.52,0.00,65.6M,"476,928"
652,ZIL,211.25,211.24,213.79,211.24,212.25,1.00,0.47%,146,1.3B,212.25,0.47%,-22.25%,30.28,0.97,306.1K,"1,135"


In [54]:
final_df.to_csv(date+".csv")
